In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn import metrics
from sklearn.metrics import accuracy_score


# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

### Implementation of SGD classifier using Dask-ML on the entire dataset.

In [2]:
import dask.dataframe as dd
import dask.array as da

# Load Data

In [4]:
df_subset = dd.read_csv('../input/train-data-modified/train_df.csv')

In [5]:
df_subset.head()

In [6]:
test_df = dd.read_csv('../input/test-data-modified/test_df.csv')

In [7]:
test_df.head()

In [8]:
main_X = df_subset.drop(['HasDetections', 'MachineIdentifier'], axis=1).to_dask_array(lengths=True)
main_y = df_subset['HasDetections'].to_dask_array(lengths=True)

In [9]:
test_X = test_df.drop(['MachineIdentifier'], axis=1).to_dask_array(lengths=True)

# Scaling

In [10]:
#SCALING NOT WORKING
# from dask_ml.preprocessing import StandardScaler
# scalar = StandardScaler()
# scalar.fit_transform(main_X)

In [11]:
pip install dask-ml[complete]

# SGD classifier

In [16]:
from dask_ml.wrappers import Incremental
from sklearn.linear_model import SGDClassifier

##### Using the best hyperparameters obtained from fine-tuning. 

In [17]:
params = {
    "loss" : ["hinge", "log"],
    "alpha" : [0.0001, 0.001, 0.01, 0.1],
    "penalty" : ["l2", "l1", "none"],
}

estimator = SGDClassifier(max_iter=500)
clf = dcv.GridSearchCV(estimator, param_grid=params)


clf = Incremental(estimator)
clf.fit(main_X, main_y, classes=[0, 1])

In [18]:
y_pred = clf.predict(test_X)
# print('Accuracy Score: ', (accuracy_score(y_test, y_pred)))
# print('ROC-AUC score: ', metrics.roc_auc_score(y_test, y_pred))

In [19]:
test_df["HasDetections"] = y_pred
to_submit = test_df[["MachineIdentifier", "HasDetections"]]
to_submit.to_csv("submitit.csv", single_file=True, index=False)

In [20]:
y_pred